In [1]:
import polars as pl

In [2]:
# Import incidents and format columns
df = pl.read_csv(
    "./data/incident_dump.csv",
).with_columns(
    pl.col("reported")
    .str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S%z")
    .dt.convert_time_zone("America/Chicago"),
    pl.col("reported_date").str.to_date("%Y-%m-%d"),
    pl.col("validated_location").str.split(",").cast(pl.List(pl.Float64)),
    pl.col("incident").str.to_lowercase(),
)

In [3]:
df.groupby(["incident"]).agg(pl.count()).sort("count", descending=True)

incident,count
str,u32
"""theft""",2953
"""information""",1365
"""found property…",914
"""lost property""",828
"""traffic violat…",533
"""information / …",481
"""liquor law vio…",440
"""medical call""",419
"""battery""",418


In [8]:
excluded_list = [
    "Fondling",
    "Medical Call",
    "Luring a Minor",
    "Lost Property",
    "Stalking",
    "Sexual Assault",
    "Dating",
    "Stalking",
    "Domestic",
    "Sex",
    "Found Property",
    "Mental Health",
    "Harassment by Electronic Means",
    "Well-Being",
    "Threatening Phone Call",
    "Medical Transport",
    "Warrant",
    "Lost Wallet",
    "Fire Alarm",
    "Chemical Spill",
    "Suspicious Mail",
    "Eavesdropping",
    "Sex Offense",
    "Sex Offender",
    "Sex Crime",
    "Domestic Aggravated Battery",
    "Dating Violence",
    "Harassing Messages",
]
df = df.filter(~pl.col("incident").str.contains("|".join(excluded_list)))
df.groupby(["incident"]).agg(pl.count()).sort("count", descending=True)

incident,count
str,u32
"""theft""",2953
"""information""",1365
"""found property…",914
"""lost property""",828
"""traffic violat…",533
"""information / …",481
"""liquor law vio…",440
"""medical call""",419
"""battery""",418


In [5]:
def list_to_parsed_set(unparsed_list):
    parsed_set = set()
    for element in unparsed_list:
        if "/" in element:
            for p in element.split("/"):
                fmt_element = p.strip().lower()
                if p:
                    parsed_set.add(fmt_element)
        else:
            fmt_element = element.strip().lower()
            parsed_set.add(fmt_element)
    return parsed_set

In [7]:
print(list_to_parsed_set(df["incident"].to_list()))

{'', 'illness', 'attempt theft from person', 'found drug paraphernalia', 'found wallet', 'well-being', 'domestic issue', 'attempt aggravated robbery', 'pedestrian', 'unlawful possession of a handgun', 'personation of peace officer', 'mental health transport', 'automobile', 'possession of marijuana', 'public way', 'attempted theft', 'aggravated domestic assault', 'mental health', 'minor personal injury', 'armed robbery', 'threat', 'unlawful discharge of firearm', 'identity theft', 'aggravated robbery', 'unlawful discharge of a firearm', 'haz mat event', 'vehicle crash', 'non-criminal complaint', 'criminal trespass to motor vehicle', 'criminal sexual assault', 'minor injury', 'weapon arrest', 'found', 'attempt home invasion', 'dating violence', 'recovered property', 'obstructing peace officer', 'vehicle recovery', 'theft of lost or mislaid property', 'delayed report', 'lost property', 'suspicious phone call', 'aggravated domestic battery', 'aggravated discharge of a firearm', 'trespass',